In [1]:
from crypto import cryptopuf
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


In [ ]:
# Function to generate CRPs and train SVM
def train_and_evaluate_svm(num_training_crps, num_test_challenges):
    print(f"Training with {num_training_crps} CRPs...")
    np.random.seed(seed)
    
    # Generate challenges for training and testing
    challenges = np.random.randint(0, 2, size=(num_training_crps, num_bits), dtype=np.uint8) * 2 - 1  # Convert to {-1, 1}
    test_challenges = np.random.randint(0, 2, size=(num_test_challenges, num_bits), dtype=np.uint8) * 2 - 1

    # Instantiate CryptoPUF
    tinyjambu_key = np.random.bytes(16).hex()  # Randomly generated 128-bit key
    tinyjambu_nonce = "000000000000000000000000"  # 12 bytes = 96 bits
    crypto_puf = cryptopuf.CryptoPUF(challenge_length=num_bits, key=tinyjambu_key, nonce=tinyjambu_nonce, crypto = 'tinyjambu', seed = seed)

    # Generate responses using CryptoPUF's eval function
    responses = crypto_puf.eval(challenges)
    test_responses = crypto_puf.eval(test_challenges)

    # Train SVM model
    svm_model = SVC(kernel='rbf', C=1.0, gamma='scale')
    svm_model.fit(challenges, responses.ravel())
    print(f"Training completed for {num_training_crps} CRPs.")

    # Predict responses using the trained SVM model
    predicted_responses = svm_model.predict(test_challenges)

    # Calculate accuracy
    accuracy = accuracy_score(test_responses, predicted_responses)
    print(f"Accuracy: {accuracy:.4f}\n")
    return accuracy

In [3]:
# Set up variables
num_bits = 128              # Number of bits in each challenge (matching block size)
num_training_crps = 5000     # Number of CRPs for training
num_test_challenges = 1000   # Number of CRPs for testing

# Train and evaluate the model once with 5,000 training CRPs
seed = 58
accuracy = train_and_evaluate_svm(num_training_crps, num_test_challenges)
print(f"Final accuracy with {num_training_crps} training CRPs: {accuracy:.4f}")

Training with 5000 CRPs...
Initializing CryptoPUF based on tinyjambu crypto core...
Training completed for 5000 CRPs.
Accuracy: 0.5010

Final accuracy with 5000 training CRPs: 0.5010
